In [1]:
import pandas as pd
import ast


# Set Pandas display options (to make it easy to just copy paste things, no cutoffs)
pd.set_option('display.max_colwidth', None)  # No limit on column width
pd.set_option('display.expand_frame_repr', True)  # Prevent wrapping to new lines

In [2]:
documents =pd.read_csv('data/satoshi_with_questions.csv')

documents.head()

,content,chunk_index,collection,content_type,filename,sequence_number,source,source_file,title,total_chunks,useless_to_recall,questions
0,"Emails\nBitcoin P2P e-cash paperOct 31, 2008, 18:10:00 UTCBitcoin P2P e-cash paperNov 3, 2008, 01:37:43 UTCBitcoin P2P e-cash paperNov 3, 2008, 16:23:49 UTCBitcoin P2P e-cash paperNov 6, 2008, 20:15:40 UTCBitcoin P2P e-cash paperNov 8, 2008, 18:54:38 UTCBitcoin P2P e-cash paperNov 9, 2008, 01:58:48 UTCBitcoin P2P e-cash paperNov 9, 2008, 03:09:49 UTCBitcoin P2P e-cash paperNov 9, 2008, 16:31:26 UTCBitcoin P2P e-cash paperNov 10, 2008, 02:14:30 UTCBitcoin P2P e-cash paperNov 10, 2008, 22:18:20 UTCBitcoin P2P e-cash paperNov 13, 2008, 22:56:55 UTCBitcoin P2P e-cash paperNov 14, 2008, 18:55:35 UTCBitcoin P2P e-cash paperNov 15, 2008, 04:43:00 UTCBitcoin P2P e-cash paperNov 15, 2008, 18:02:00 UTCBitcoin P2P e-cash paperNov 17, 2008, 17:24:43 UTCBitcoin v0.1 releasedJan 8, 2009, 19:27:40 UTCBitcoin v0.1 releasedJan 16, 2009, 16:03:14 UTCBitcoin v0.1 releasedJan 25, 2009, 15:47:10 UTC",0,emails,email,email_002_Cryptography_Mailing_List,2,nakamotoinstitute_files\emails\email_002_Cryptography_Mailing_List.html,nakamotoinstitute_files\emails\email_002_Cryptography_Mailing_List.html,Cryptography Mailing List,1,False,"[""What is mentioned with regards to Bitcoin's version number in these emails?"", 'Was Bitcoin v0.1 released on Jan 8, 2009?', 'When was the v0.1 release of Bitcoin?']"
1,"Emails\n[p2p-research] Bitcoin open source implementation of P2P currencyFeb 11, 2009, 22:37:54 UTC[p2p-research] Bitcoin open source implementation of P2P currencyFeb 12, 2009, 19:08:24 UTC[p2p-research] Bitcoin open source implementation of P2P currencyFeb 13, 2009, 02:38:20 UTC[p2p-research] Bitcoin open source implementation of P2P currencyFeb 13, 2009, 18:36:45 UTC[p2p-research] Bitcoin P2P e-currency v0.1 releasedFeb 13, 2009, 18:52:41 UTC",0,emails,email,email_004_P2P_Research_List,4,nakamotoinstitute_files\emails\email_004_P2P_Research_List.html,nakamotoinstitute_files\emails\email_004_P2P_Research_List.html,P2P Research List,1,False,"[""What was the release date of Bitcoin's open source implementation of P2P currency?"", 'Did Satoshi Nakamoto release a version of Bitcoin on February 13, 2009?', 'What specific features or milestones did Bitcoin v0.1 introduce?']"
2,"From: | Satoshi Nakamoto | Subject: | Bitcoin P2P e-cash paper | Date: | November 6, 2008 at 20:15:40 UTC\n>[Lengthy exposition of vulnerability of a systm to use-of-force\n>monopolies ellided.]\n>\n>You will not find a solution to political problems in cryptography.\nYes, but we can win a major battle in the arms race and gain a new territory of freedom for several years.\nGovernments are good at cutting off the heads of a centrally controlled networks like Napster, but pure P2P networks like Gnutella and Tor seem to be holding their own.\nSatoshi\n---------------------------------------------------------------------\nThe Cryptography Mailing List\nUnsubscribe by sending ""unsubscribe cryptography"" to majordomo at metzdowd.com\nBitcoin P2P e-cash paper\nCryptography Mailing List",0,emails,email,email_010_Bitcoin_P2P_e-cash_paper,10,nakamotoinstitute_files\emails\email_010_Bitcoin_P2P_e-cash_paper.html,nakamotoinstitute_files\emails\email_010_Bitcoin_P2P_e-cash_paper.html,Bitcoin P2P e-cash paper,1,False,"[""What is Satoshi Nakamoto's view on using cryptography to solve political problems?"", 'Did Satoshi mention the vulnerability of centrally controlled networks and P2P networks like Napster, Gnutella, and Tor?', 'What major battle in the arms race did Satoshi Nakamoto foresee being won by cryptographers?']"
3,"From: | Satoshi Nakamoto | Subject: | [bitcoin-list] Welcome | Date: | December 10, 2008 at 17:00:23 UTC\nWelcome to the Bitcoin mailing list!\n[bitcoin-list] Welcome\nbitcoin-list",0,emails,email,email_022_[bitcoin-list]_Welcome,22,nakamotoinstitute_files\emails\email_022_[bitcoin-list]_Welcome.html,nakamotoinstitute_files\emails\email_022

In [9]:
documents.questions.iloc[0][0]

'['

In [ ]:
# Common problem I had in GraphViewer project as well: Once the list of strings were stored as csv it was a a string.
# So we need to parse it back to a list of strings.

ast.literal_eval(documents.questions.iloc[0])

["What is mentioned with regards to Bitcoin's version number in these emails?",
 'Was Bitcoin v0.1 released on Jan 8, 2009?',
 'When was the v0.1 release of Bitcoin?']

In [3]:
# Helper function from Grok 
# there were some malformed strings that cause regular literal_eval to fail.
def safe_literal_eval(s):
    """Safely evaluate a string as a Python literal, with fallback options."""
    try:
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        # Try to fix common issues
        try:
            # Replace double quotes with single quotes for consistency
            fixed_s = s.replace('""', "'").replace('"', "'")
            return ast.literal_eval(fixed_s)
        except (ValueError, SyntaxError):
            try:
                # Try to parse as JSON if it's valid JSON
                import json
                return json.loads(s)
            except (ValueError, json.JSONDecodeError):
                # Last resort: return the string as a single-item list
                print(f"Warning: Could not parse: {s[:50]}...")
                return [str(s)]

documents['questions'] = documents['questions'].apply(safe_literal_eval)

  Invali...
  Invali...


## Manually inspect exact duplicates

In [4]:
# documents['questions'] = documents['questions'].apply(ast.literal_eval)


all_questions = []
all_questions_index = [] # New variable to store the original row index for each question
for i, question_entry in enumerate(documents['questions']):
    # The 'questions' column is expected to contain lists of strings after
    # the safe_literal_eval application. However, if there were issues
    # or if some entries remained as string representations of lists,
    # `extend()` would iterate over characters of the string.
    # To ensure robustness, we re-apply `safe_literal_eval` to each entry.
    # This guarantees that `parsed_questions` is always a list of strings,
    # even if `question_entry` was a string representation of a list,
    # or a single unparseable string (which safe_literal_eval wraps in a list).
    # We cast to `str()` first, as `safe_literal_eval` expects a string input.
    parsed_questions = safe_literal_eval(str(question_entry))
    all_questions.extend(parsed_questions)
    # For each question added, record its original row index
    all_questions_index.extend([i] * len(parsed_questions)) # to add an entry for every question in the list


# Calculate statistics
total_questions = len(all_questions)
unique_questions = len(set(all_questions))
duplicate_count = total_questions - unique_questions

print(f"Total questions: {total_questions}")
print(f"Unique questions: {unique_questions}")
print(f"Duplicate questions: {duplicate_count}") # "Hardcore" duplicates 


Total questions: 3601
Unique questions: 3573
Duplicate questions: 28


In [5]:
# Build a dataframe of all questions and their originating row index
df_q = pd.DataFrame({
    'question': all_questions,
    'doc_index': all_questions_index
})

# Keep all rows whose question appears more than once
duplicate_questions_df = df_q[df_q.duplicated('question', keep=False)].copy()

# Fetch the corresponding content using positional index (matches enumerate)
duplicate_questions_df['content'] = duplicate_questions_df['doc_index'].apply(lambda i: documents.iloc[i]['content'])

# Optional: sort for easier review
duplicate_questions_df = duplicate_questions_df.sort_values(['question', 'doc_index']).reset_index(drop=True)

duplicate_questions_df

,question,doc_index,content
0,,206,"Right, that is quite a bit better.Can you give me any examples of other stuff that does it that way?Â (and what the command line looks like)The main change you're talking about here is instead of -rpcpw= when you start bitcoind, you'd use a switch that specifies a text file to go and read it from, right?Â (any ideas what I should name the switch?)\nRe: JSON-RPC password\nJuly 19, 2010 at 04:43:13 UTC\nBitcoinTalk"
1,,835,"CPU power gangs up on one.Â Instead, all networks in the world would share combined CPU power, increasing the total strength.Â It would make it easier for small networks to get started by tapping into a ready base of miners.\nRe: BitDNS and Generalizing Bitcoin\nDecember 9, 2010 at 21:02:42 UTC\nBitcoinTalk"
2,Did Satoshi Nakamoto discuss peer-to-peer networks in this email?,115,"davidonpda, were you also running laszlo's build previously?Check if the ""%appdata%"" directory exists, and ""%appdata%\bitcoin""Try:rename ""%appdata%\bitcoin"" bitcoin2does it work then?\nRe: 0.3 almost ready\nJune 22, 2010 at 19:25:13 UTC\nBitcoinTalk"
3,Did Satoshi Nakamoto discuss peer-to-peer networks in this email?,797,"Code:diff -u old\main.cpp new\main.cpp--- old\main.cppSun Oct 03 20:57:20 2010+++ new\main.cppSun Oct 03 20:57:54 2010@@ -2831,6 +2831,10 @@bool fUseSSE2 = ((fIntel && nFamily * 10000 + nModel >=Â 60026) ||(fAMDÂ Â && nFamily * 10000 + nModel >= 160010));+Â Â // AMD reports a lower model number in 64-bit mode+Â Â if (fAMD && sizeof(void*) > 4 && nFamily * 10000 + nModel >= 160004)+Â Â Â Â fUseSSE2 = true;+static bool fPrinted;if (!fPrinted){@@ -2989,6 +2993,17 @@// Transaction fee based on block sizeint64 nMinFee = tx.GetMinFee(nBlockSize);+Â Â Â Â Â Â Â Â Â Â //////// temporary code+Â Â Â Â Â Â Â Â Â Â if (nBlockSize < MAX_BLOCK_SIZE_GEN / 10 && GetWarnings(""statusbar"") == """")+Â Â Â Â Â Â Â Â Â Â {+Â Â Â Â Â Â Â Â Â Â Â Â if (nBestHeight < 91000)+Â Â Â Â Â Â Â Â Â Â Â Â Â Â nMinFee = 0;+Â Â Â Â Â Â Â Â Â Â Â Â if (nBestHeight < 100000 && nTxSize < 2000)+Â Â Â Â Â Â Â Â Â Â Â Â Â Â nMinFee ="
4,What did Satoshi Nakamoto say about micropayments?,545,". We have to trust them with\nour privacy, trust them not to let identity thieves drain our accounts.\nTheir massive overhead costs make micropayments impossible.\nA generation ago, multi-user time-sharing computer systems had a similar\nproblem. Before strong encryption, users had to rely on password\nprotection to secure their files, placing trust in the system\nadministrator to keep their information private. Privacy could always\nbe overridden by the admin based on his judgment call weighing the\nprinciple of privacy against other concerns, or at the behest of his\nsuperiors. Then strong encryption became available to the masses, and\ntrust was no longer required. Data could be secured in a way that was\nphysically impossible for others to access, no matter for what reason,\nno matter how good the excuse, no matter what.\nIt's time we had the same thing for money"
5,What did Satoshi Nakamoto say about micropayments?,727,"Forgot to add the good part about micropayments. Â While I don't think Bitcoin is practical for smaller micropayments right now, it will eventually be as storage and bandwidth costs continue to fall. Â If Bitcoin catches on on a big scale, it may already be the case by that time. Â Another way they can become more practical is if I implement client-only mode and the number of network nodes consolidates into a smaller number of professional server farms. Â Whatever size micropayments you need will eventually be practical. Â I think in 5 or 10 years, the bandwidth and storage will seem trivial.I am not claiming that the network is impervious to DoS attack. Â I think most P2P networks can be DoS attacked in numerous ways"
6,What did Satoshi Nakamoto say about peer-to-peer networks?,17,"Could be. They're talking about the old Chaumian central mint stuff, but maybe only because that was the only thing availabl

## Using Embeddings and Cosine Similarity

In [6]:
from dotenv import load_dotenv
load_dotenv()

# from langfuse.openai import AsyncOpenAI
from openai import AsyncOpenAI
from langfuse import observe

from diskcache_decorator import cached

In [7]:
client = AsyncOpenAI(
    base_url="http://localhost:11434/v1",  # Ollama's default local endpoint
    api_key="ollama"  # Dummy key, required but not used by Ollama
)

# EMBED_MODEL = "mxbai-embed-large:latest"
EMBED_MODEL = "mxbai-embed-large:335m"

In [4]:
# test call

# note that the main difference is we provide "input" instead of "message"
embedding = await client.embeddings.create(
    input="Hello, world!",
    model=EMBED_MODEL,
)

embedding.data[0].embedding[0]  # first numer to not spam screen

0.017286297

In [8]:
# @observe(as_type="generation")
@cached()
async def make_cached_traced_embedding(input):
    embedding = await client.embeddings.create(
    input=input,
    model=EMBED_MODEL,
)
    return embedding.data[0].embedding

In [6]:
embedding = await make_cached_traced_embedding("blubb")
embedding[0]

0.010233218

### Embed entire dataset

In [29]:
def make_dataset(id_document, id_question, question, embedding=None):
    return {
        "id_document": id_document,
        "id_question": id_question,
        "question": question,
        "embedding": embedding
    }

questions_with_embeddings = []
index_for_questions_with_embeddings = {}

# Reminder: within each document, questions are a list of strings, we can index into it.
for i, row in documents.iterrows():
    index_for_questions_with_embeddings[i] = {} # create a nested dict so we can access document/question pairs with double index.
    for q_idx, question in enumerate(row['questions']):
        _ds = make_dataset(i, q_idx, question)
        index_for_questions_with_embeddings[i][q_idx] = _ds
        questions_with_embeddings.append(_ds)

In [ ]:
# Now we have two data structures:
# 1. questions_with_embeddings: a list of dicts, each dict contains the question and its embedding. easy to iterate over.
# 2. index_for_questions_with_embeddings: full information about a document. double indexing to access doc/question pair easily.

print(questions_with_embeddings[0])
print(index_for_questions_with_embeddings[0])

{'id_document': 0, 'id_question': 0, 'question': "What is mentioned with regards to Bitcoin's version number in these emails?", 'embedding': None}
{0: {'id_document': 0, 'id_question': 0, 'question': "What is mentioned with regards to Bitcoin's version number in these emails?", 'embedding': None}, 1: {'id_document': 0, 'id_question': 1, 'question': 'Was Bitcoin v0.1 released on Jan 8, 2009?', 'embedding': None}, 2: {'id_document': 0, 'id_question': 2, 'question': 'When was the v0.1 release of Bitcoin?', 'embedding': None}}


In [44]:
# See also notebook 5 where we used batching.
import traceback
import asyncio

concurrency_limit = 20  # tune as needed
sem = asyncio.Semaphore(concurrency_limit)

async def do_task(ds):
    try:
        async with sem:
            # mutating this slice will update the original questions_with_embeddings list.
            # The function then just returns none, which means it worked.
            ds['embedding'] = await make_cached_traced_embedding(ds['question'])
    except Exception as e:
        return (ds, e, traceback.format_exc())


tasks = [do_task(ds) for ds in questions_with_embeddings] # test with [:50] or so first
results = await asyncio.gather(*tasks)



In [45]:
import pickle

# Save both variables in a single pickle file
data_to_save = {
    'questions_with_embeddings': questions_with_embeddings,
    'index_for_questions_with_embeddings': index_for_questions_with_embeddings
}

with open('embeddings_data.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

print("Both variables saved to single pickle file!")
print(f"Saved {len(questions_with_embeddings)} question embeddings")

Both variables saved to single pickle file!
Saved 3601 question embeddings


#### Reload data here:

In [9]:
import pickle

# Load both variables from the single pickle file
with open('embeddings_data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

# Extract the variables
questions_with_embeddings = loaded_data['questions_with_embeddings']
index_for_questions_with_embeddings = loaded_data['index_for_questions_with_embeddings']

print("Both variables loaded from pickle file!")
print(f"Loaded {len(questions_with_embeddings)} question embeddings")

Both variables loaded from pickle file!
Loaded 3601 question embeddings


### Calculate Cosine Similarity

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [11]:
def get_cosine_similarity(embedding1, embedding2):
    """
    scikit-learn functions usually expect a 2D array (1,n) instead of (n,)
    we have to turn it into a numpy array and reshape it. 
    (1,-1) means it will have 1 row, and n columns (-1 means it automatically calculates n based on the array lenght).
    [1,2,3] --> [[1,2,3]]
    instead of that I could probably join the two arrays and calculate the pairwise cos-sim, get a 2x2 matrix (where its 1 on the diagonal). but here i just want a single value.
    """
    to_np = lambda e: np.array(e).reshape(1, -1)
    embedding1, embedding2 = to_np(embedding1), to_np(embedding2)
    _cos_sim = cosine_similarity(embedding1, embedding2)
    return _cos_sim[0][0]

In [22]:
get_cosine_similarity(embedding, embedding)

np.float64(0.9999999999999993)

We already checked for exact duplicates when parsing the questions. This is just another way of doing it...

In [12]:
from operator import itemgetter

def find_exact_duplicates(questions_with_embeddings):
    get_question = itemgetter('question')
    seen = set()
    exact_duplicates = set()
    for q in map(get_question, questions_with_embeddings):
        if q in seen:
            exact_duplicates.add(q)
        else:
            seen.add(q)
    return exact_duplicates

"""
version above with itemgetter is just faster and fancier way of doing this:

def find_exact_duplicates(questions_with_embeddings):
    seen, dups = set(), set()
    for x in questions_with_embeddings:
        q = x['question']
        if q in seen:
            dups.add(q)
        else:
            seen.add(q)
    return dups
"""

exact_duplicates = find_exact_duplicates(questions_with_embeddings)
exact_duplicates

{'',
 'Did Satoshi Nakamoto discuss peer-to-peer networks in this email?',
 'What did Satoshi Nakamoto say about micropayments?',
 'What did Satoshi Nakamoto say about peer-to-peer networks?',
 "What is Satoshi Nakamoto's view on peer-to-peer networks?",
 "What is Satoshi's view on peer-to-peer networks?",
 "What was Satoshi Nakamoto's view on peer-to-peer networks?",
 "What was Satoshi's view on peer-to-peer networks?",
 ']'}

### Next: Within each document, find questions that are roughly the same.

In [ ]:
# Just an experiment, not used, but to illustrate we want these combinations.
for doc_id in index_for_questions_with_embeddings:
    for question_id in index_for_questions_with_embeddings[doc_id]:
        print(doc_id, question_id)
    break

0 0
0 1
0 2


May look a bit confusing below, try to print out individual elements of these data structures, then it will be clear what we are iterating over.

In [13]:
from itertools import combinations

question_combinations_per_document = {}
question_combinations_per_document_similarity = {}

for doc_id in index_for_questions_with_embeddings:
    # worthwile to look at documentation of combinations.
    # we iterate over the keys of each dictionary (remember each document in the list has a dictionaries for each question), the 2 indicate it will make pairwise combinations.
    question_combinations_per_document[doc_id] = list(combinations(index_for_questions_with_embeddings[doc_id].keys(), 2))
    # Now we use the same structure with lists of combinations but fill in the corresponding similarity scores.
    question_combinations_per_document_similarity[doc_id] = []
    for (q1, q2) in question_combinations_per_document[doc_id]:   # parentheses are optional. indicate that we have tuples here.
        e1 = index_for_questions_with_embeddings[doc_id][q1]['embedding']
        e2 = index_for_questions_with_embeddings[doc_id][q2]['embedding']
        sim = get_cosine_similarity(e1, e2)
        question_combinations_per_document_similarity[doc_id].append(sim)


In [60]:
for (q1, q2), sim in zip(question_combinations_per_document[0], question_combinations_per_document_similarity[0]):
    print(q1, q2, sim)

0 1 0.6864653957328704
0 2 0.7297580536314496
1 2 0.8621906991184569


In [14]:
from pprint import pprint  # simply puts each input on a new line. (and manages indentation or line breaks)

# Has some problems when there is only one or no questions...?
# for doc_id in question_combinations_per_document_similarity:
#     for (q1, q2), similarity_score in zip(question_combinations_per_document[0], question_combinations_per_document_similarity[0]):
#         # Above gives e.g.: 0 1 0.6864653957328704 \n  0 2 0.7297580536314496  \n  1 2 0.8621906991184569
#         # Now also get the corresponding questions for review.
#         q1_question = index_for_questions_with_embeddings[doc_id][q1]['question']
#         q2_question = index_for_questions_with_embeddings[doc_id][q2]['question']
#         if similarity_score > 0.8:
#             # print(similarity_score, q1_question, q2_question)
#             pprint((f"{similarity_score:.2f}", q1_question, q2_question)) # pprint expects a single argument. thus the double parentheses.
#             print() # to add a line break after each entry.



for doc_id in question_combinations_per_document_similarity:
    combos = question_combinations_per_document[doc_id]
    sims = question_combinations_per_document_similarity[doc_id]

    for (q1, q2), similarity_score in zip(combos, sims):
        # Above gives e.g.: 0 1 0.6864653957328704 \n  0 2 0.7297580536314496  \n  1 2 0.8621906991184569
        # Now also get the corresponding questions for review.
        doc_idx = index_for_questions_with_embeddings[doc_id]
        # optional guard if keys might be missing
        if q1 not in doc_idx or q2 not in doc_idx:
            continue
        q1_question = doc_idx[q1]['question']
        q2_question = doc_idx[q2]['question']
        if similarity_score > 0.86 and q1_question != q2_question: # we already looked at exact duplicates. to keep it shorter here.
            pprint((f"{similarity_score:.2f}", q1_question, q2_question)) # pprint expects a single argument. thus the double parentheses.
            print() # to add a line break after each entry.


('0.86',
 'Was Bitcoin v0.1 released on Jan 8, 2009?',
 'When was the v0.1 release of Bitcoin?')

('0.87',
 'Who asked Satoshi Nakamoto for help compiling and installing Bitcoin on '
 'Linux?',
 "How did the community respond to Satoshi's request for instructions on "
 'compiling and installing Bitcoin on Linux?')

('0.90',
 'What options did users of wxWidgets use to build or configure the software?',
 'Can you explain some of the options used for building or configuring '
 'wxWidgets?')

('0.86',
 'Was malloc.h necessary to include before committing changes to headers.h for '
 'Satoshi?',
 'At what point did Satoshi commit the changes to headers.h?')

('0.86',
 'Can the bitcoin server receive payments via a single-use bitcoin address?',
 'Did Satoshi mention any specific technical details about how the server '
 'handles payments to single-use bitcoin addresses?')

('0.87',
 'Will a shop owner be automatically notified when a payment is received via a '
 'single-use bitcoin address?'

In [57]:
question_combinations_per_document_similarity

{0: [np.float64(0.6864653957328704),
  np.float64(0.7297580536314496),
  np.float64(0.8621906991184569)],
 1: [np.float64(0.7372945700109843),
  np.float64(0.7226673041572994),
  np.float64(0.6691826795416889)],
 2: [np.float64(0.6252358060466829),
  np.float64(0.7535434769399083),
  np.float64(0.6033944987985799)],
 3: [np.float64(0.5412407430729286),
  np.float64(0.6323084044868046),
  np.float64(0.7095599369844926),
  np.float64(0.4661963160794194),
  np.float64(0.5927520720184083),
  np.float64(0.6115515713998052),
  np.float64(0.6771574094335887),
  np.float64(0.6965797505867608),
  np.float64(0.4848398514987524),
  np.float64(0.5736278494777782)],
 4: [np.float64(0.7530179182813821),
  np.float64(0.5949991175785313),
  np.float64(0.5752895572678625),
  np.float64(0.6373704021917608),
  np.float64(0.677849364565931),
  np.float64(0.604115347625484),
  np.float64(0.7091644505872232),
  np.float64(0.6774494789736804),
  np.float64(0.5780927371619429),
  np.float64(0.5555108873281875

## Bad Question List

and lists for questions too keep or remove irrespective of similarity.

In [21]:
BAD_QUESTIONS = [
    "What is Satoshi's view on peer-to-peer networks?",
    "Does the bitcoin would reduce with the development by these zombie farms or generate?",
    "When did Satoshi propose this change?",
    "Was a special case implemented later on in Bitcoin's development for handling this specific problem.",
    "Are there any details about peer-to-peer networks mentioned in this particular email exchange of Satoshi Nakamoto?"
]

Now I am trying on my own to achieve the following:
- Calculate the embedding for each bad question.
- Find any questions in my full question dataset that is similar to ANY of these bad questions.

In [22]:
import asyncio

def make_bad_question_dataset(id_bad_question, question, embedding=None):
    return {
        "id_bad_question": id_bad_question,
        "question": question,
        "embedding": embedding
    }

# Create initial datasets without embeddings
bad_questions_datasets = [
    make_bad_question_dataset(i, question)
    for i, question in enumerate(BAD_QUESTIONS)
]

async def do_embedding_task(ds):
    ds['embedding'] = await make_cached_traced_embedding(ds['question'])


# Create tasks
tasks = [do_embedding_task(ds) for ds in bad_questions_datasets]

# Run them concurrently
await asyncio.gather(*tasks)

[None, None, None, None, None]

In [23]:
bad_questions_datasets

[{'id_bad_question': 0,
  'question': "What is Satoshi's view on peer-to-peer networks?",
  'embedding': [0.0063467226,
   0.013246146,
   -0.0014099213,
   0.003525518,
   0.013385681,
   -0.027279047,
   -0.023863476,
   0.0019541604,
   0.024391416,
   0.07915652,
   -0.01678232,
   -0.021309074,
   0.047633987,
   0.005249657,
   -0.058849577,
   0.026683338,
   -0.03407936,
   -0.01633909,
   -0.036947787,
   -0.03846447,
   0.029030273,
   -0.026414385,
   0.004246949,
   -0.017686471,
   -0.0030385423,
   0.024608457,
   -0.006795438,
   0.029426884,
   0.112801075,
   0.04898702,
   0.03660623,
   -0.0135991005,
   0.0030092625,
   -0.02445392,
   0.007856273,
   0.002904299,
   0.023001961,
   -0.011470684,
   -0.00081890903,
   -0.03426634,
   0.035287246,
   -0.0075033824,
   0.09439172,
   -0.051653825,
   -0.04749214,
   0.013744788,
   -0.0063909586,
   -0.046183754,
   -0.016864508,
   -0.06178227,
   -0.036456455,
   0.017373629,
   0.035625428,
   -0.04202166,
   0.047

In [32]:
index_for_questions_with_embeddings[0][1]['embedding']  # I think i will update this, as its more convenient with double indexing.

[0.009601309,
 -0.008537122,
 0.00077446946,
 -0.00021934048,
 0.02977927,
 -0.027031047,
 0.0033692378,
 -0.019547846,
 -0.010279523,
 0.026084121,
 0.0034205306,
 -0.029624743,
 -0.006279386,
 0.0010623214,
 -0.018585207,
 0.0081440285,
 -0.006339836,
 0.002691504,
 -0.0065372065,
 0.020811342,
 0.013721432,
 0.001866162,
 0.005924683,
 -0.022215785,
 -0.014877686,
 0.05773007,
 0.019214312,
 0.007489409,
 0.06922635,
 0.07549322,
 0.041468702,
 -0.042138584,
 0.04456905,
 -0.020592073,
 -0.031673215,
 -0.008402288,
 0.010357801,
 -0.0043394733,
 -0.016293677,
 -0.010677792,
 -0.01684843,
 -0.015832832,
 0.04088214,
 -0.00538268,
 -0.08839199,
 0.01627173,
 0.008784507,
 -0.0025537543,
 0.0037155761,
 -0.024622459,
 -0.012896857,
 -0.027614763,
 0.008216032,
 0.007125563,
 0.015666656,
 -0.0056001768,
 -0.013137973,
 -0.028550237,
 -0.028549727,
 0.033946063,
 0.022175657,
 -0.0019414346,
 0.06166508,
 -0.0771245,
 0.012940258,
 0.024130287,
 -0.024524702,
 0.018039802,
 -0.005971511

In [28]:
questions_with_embeddings[0]

{'id_document': 0,
 'id_question': 0,
 'question': "What is mentioned with regards to Bitcoin's version number in these emails?",
 'embedding': [0.038117148,
  -0.017908264,
  0.0085548535,
  0.01923597,
  -0.039248545,
  -0.026890658,
  0.038319916,
  -0.003432576,
  -0.015596412,
  0.0549171,
  0.0070316615,
  -0.0033220055,
  0.008513049,
  0.006123755,
  -0.066647865,
  0.0077873054,
  -0.015732292,
  0.0006298338,
  0.010419241,
  -0.001275963,
  -0.0013277204,
  0.011682249,
  -0.007752757,
  0.023941655,
  0.0073712403,
  0.015634194,
  -0.013991731,
  -0.0016760726,
  0.071689926,
  0.07777467,
  -0.010472149,
  -0.012658221,
  -0.0025649196,
  -0.025737505,
  -0.03650063,
  -0.0054585193,
  0.040395927,
  0.0037740387,
  -0.0014355422,
  -0.03768711,
  0.0006371587,
  -0.021094497,
  0.018108277,
  -0.018534068,
  -0.11034316,
  -0.01426587,
  -0.01522528,
  -0.0072621736,
  -0.016535107,
  -0.046274472,
  0.016943317,
  -0.029651994,
  0.06328213,
  -0.023329454,
  0.02816462

In [ ]:
"""
For each document, then each question in the full dataset:
Loop over every bad question, get cos-sim, store that value and bad question id, if the next bad question has a higher value, update the value and id.
We extend the index_for_questions_with_embeddings dataset with new keys "cos_bad_question" and "bad_question_id", later can decide on a threshold value.
"""

for doc_idx in range(len(index_for_questions_with_embeddings)):
    for q_idx in range(len(index_for_questions_with_embeddings[doc_idx])):
        question_ds = index_for_questions_with_embeddings[doc_idx][q_idx]
        
        max_sim = -1  # Assuming cosine similarity ranges from -1 to 1, initialize low
        max_id = None
        
        for bad_ds in bad_questions_datasets:
            sim = get_cosine_similarity(question_ds['embedding'], bad_ds['embedding'])
            if sim > max_sim:
                max_sim = sim
                max_id = bad_ds['id_bad_question']
        
        question_ds['cos_bad_question'] = max_sim
        question_ds['bad_question_id'] = max_id

# Deduplicate questions with LLM call

In [ ]:
from textwrap import dedent
from jinja2 import Template
from pydantic import BaseModel
from enum import Enum


class DuplicateVerdictEnum(Enum):  #seems like typing is needed, otherwise get error from pydantic
	is_duplicate="same"
	is_not_duplicate="different"

class DuplicateVerdict(BaseModel):
	concise_reasoning: str
	verdict: DuplicateVerdictEnum


_question_check_system_prompt_template = Template(dedent(
    """
    Your task is to assess if two questions are close enough to generate the same answers.

    You'll answer in JSON by respecting the following schema:
    ```ts
    {
        concise_reasoning: str
        verdict: "same" | "different"
    }

"""
))

question_check_prompt_template = Template(dedent(
	"""
    Do those two questions likely generate the same answer and could be considered duplicates?
	
	<question 1>
	{{ question1 }}
	</question 1>
	
	<question 2>
	{{ question2 }}
	</question 2>
	
	First concisely reason step by step, then provide your final verdict.
"""
))

In [22]:
# Settings and helper functions

LLM_MODEL = "deepseek-r1:8b"


def _msg(role, content):
    # simple helper function to create a message object
    return {"role": role, "content": content}

def system(content):
    return _msg("system", content)

def user(content):
    return _msg("user", content)

def assistant(content):
    return _msg("assistant", content)

@cached()
async def get_completion(messages, response_format):
    completion = await client.chat.completions.parse(
        model=LLM_MODEL,
        messages=messages,
        response_format=response_format
    )
    return completion.choices[0].message.parsed

In [27]:
# Test run

system_prompt = _question_check_system_prompt_template.render()
# print(system_prompt)
user_prompt = question_check_prompt_template.render(
    question1="What is the capital of France?",
    question2="Of which country is Paris the capital?"
)
# print(user_prompt)

messages = [
    system(system_prompt),
    user(user_prompt)
]
pprint(messages)

completion = await get_completion(messages, response_format=DuplicateVerdict)

pprint(completion)

[{'content': '\n'
             'Your task is to assess if two questions are close enough to '
             'generate the same answers.\n'
             '\n'
             "You'll answer in JSON by respecting the following schema:\n"
             '```ts\n'
             '{\n'
             '    concise_reasoning: str\n'
             '    verdict: "same" | "different"\n'
             '}\n',
  'role': 'system'},
 {'content': '\n'
             '    Do those two questions likely generate the same answer and '
             'could be considered duplicates?\n'
             '\n'
             '\t<question 1>\n'
             '\tWhat is the capital of France?\n'
             '\t</question 1>\n'
             '\n'
             '\t<question 2>\n'
             '\tOf which country is Paris the capital?\n'
             '\t</question 2>\n'
             '\n'
             '\tFirst concisely reason step by step, then provide your final '
             'verdict.',
  'role': 'user'}]
DuplicateVerdict(concise_reaso